In [33]:
import numpy as np 
import pandas as pd
import mysql.connector
from textblob import TextBlob

In [34]:
cnx = mysql.connector.connect(user='root', password='',
                              host='localhost',
                              database='yelp_db')
cursor = cnx.cursor()

In [35]:
df = pd.read_sql('SELECT text FROM yelp_db.review limit 100', con=cnx)

In [36]:
# df = pd.read_csv('review_category_user.csv')

In [37]:
# features: polarity, subjectivity 
def senAnalyze(df):
    polarity = []
    subjectivity = []
    rowNum = df.shape[0]
    for index in range(rowNum):
        analysis = TextBlob(df['text'][index])
        sen = analysis.sentiment
        polarity.append(sen.polarity)
        subjectivity.append(sen.subjectivity)
    return polarity, subjectivity
senAnalyze(df)

([0.4285714285714286,
  1.0,
  0.8333333333333334,
  0.15000000000000002,
  0.38333333333333336,
  0.3541396103896104,
  0.18906250000000002,
  0.1488095238095238,
  -0.08580542264752788,
  -0.01249999999999999,
  0.31456642316017314,
  0.3912337662337662,
  0.3580128205128205,
  0.02107142857142857,
  0.06722222222222222,
  0.85,
  -0.15000000000000002,
  0.2895833333333333,
  -0.09537037037037037,
  0.10249999999999998,
  0.22954545454545452,
  0.2474074074074074,
  0.6041666666666666,
  0.16041666666666665,
  0.2990625,
  0.004761904761904765,
  0.2811755952380952,
  0.05943181818181818,
  -0.29666666666666663,
  0.2636363636363636,
  0.28694444444444434,
  0.3371428571428571,
  0.0,
  0.6776785714285714,
  0.595,
  0.33571428571428574,
  0.6,
  0.33072977158343014,
  0.35714285714285715,
  0.5,
  0.5401515151515152,
  -0.1144047619047619,
  0.4663265306122449,
  0.1939102564102564,
  0.3814814814814815,
  0.5619999999999999,
  -0.11499999999999999,
  0.18003246753246754,
  -0.1875,

In [38]:
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [39]:
def dataClean(sentence):
    ps = PorterStemmer()
    stop_words = nltk.corpus.stopwords.words('english')
    tokens = word_tokenize(sentence)
    sentence = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', sentence)
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    filtered_tokens = [token for token in tokens if token not in stop_words]
    new_tokens = []
    for new_token in filtered_tokens:
        if new_token not in punctuations:
            new_token = ps.stem(new_token)
            new_tokens.append(new_token)
    return new_tokens

In [40]:
dataClean(df['text'][0])

['red',
 'white',
 'bleu',
 'salad',
 'super',
 'yum',
 'great',
 'addit',
 'menu',
 'thi',
 'locat',
 'clean',
 'great',
 'servic',
 'food',
 'serv',
 'right',
 'temp',
 'kid',
 'pizza',
 'alway',
 'hit',
 'lot',
 'great',
 'side',
 'dish',
 'option',
 'kiddo',
 'when',
 'I',
 "'m",
 'side',
 'town',
 'definit',
 'spot',
 'I',
 "'ll",
 'hit']

In [41]:
def features(df):
    words = []
    for index in range(df.shape[0]):
        tokens = dataClean(df['text'][index])
        for token in tokens:
            words.append(token.lower())
    
    words = nltk.FreqDist(words)
    return words.most_common(1000)

In [42]:
features(df)

[('i', 375),
 ("n't", 90),
 ('the', 86),
 ('...', 66),
 ("'s", 52),
 ('place', 48),
 ('one', 47),
 ('time', 47),
 ('good', 44),
 ('we', 44),
 ('like', 41),
 ('get', 41),
 ('food', 40),
 ('go', 39),
 ('great', 38),
 ('wait', 38),
 ('would', 36),
 ('love', 31),
 ('they', 29),
 ('nice', 29),
 ('servic', 28),
 ('back', 28),
 ('order', 28),
 ('come', 27),
 ('day', 26),
 ('got', 24),
 ('want', 24),
 ('ask', 22),
 ('realli', 22),
 ('it', 21),
 ('went', 20),
 ('also', 20),
 ('made', 19),
 ('my', 19),
 ('anoth', 19),
 ('tri', 19),
 ("'m", 18),
 ('price', 18),
 ('know', 18),
 ('even', 18),
 ('take', 18),
 ('came', 18),
 ('hour', 17),
 ('told', 17),
 ('well', 17),
 ('amaz', 16),
 ('us', 16),
 ('look', 16),
 ('could', 16),
 ('call', 16),
 ('roll', 16),
 ('make', 16),
 ("'ve", 16),
 ('say', 16),
 ('nail', 16),
 ('2', 16),
 ('littl', 16),
 ('super', 15),
 ('thi', 15),
 ('restaur', 15),
 ('room', 15),
 ('never', 15),
 ('two', 15),
 ('menu', 14),
 ('alway', 14),
 ('star', 14),
 ('said', 14),
 ('still'

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
def textVectorizer(df):
    vectorizer = CountVectorizer(min_df=1)
    textVector = vectorizer.fit_transform(list(df['text']))
    return textVector.todense()
textVectorizer(df)

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [48]:
def biTextVectorizer(df):
    vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
    textVectorizer = vectorizer.fit_transform(df['text'])
    return textVector.todense()
biTextVectorizer(df)

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [45]:
def tfidfVectorizer(df):
    vectorizer = TfidfVectorizer()
    textVector = vectorizer.fit_transform(list(df['text']))
    return textVector.todense()
tfidfVectorizer(df)

matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [46]:
def textLength(df):
    rowNum = df.shape[0]
    sentenceLen = []
    for index in range(rowNum):
        tokens = dataClean(df['text'][index])
        sentenceLen.append(len(tokens))
    return sentenceLen
textLength(df)

[38,
 14,
 13,
 15,
 23,
 32,
 23,
 93,
 189,
 133,
 199,
 37,
 45,
 80,
 197,
 15,
 35,
 28,
 86,
 58,
 43,
 107,
 14,
 66,
 89,
 67,
 17,
 374,
 20,
 44,
 125,
 48,
 94,
 19,
 14,
 33,
 42,
 135,
 30,
 46,
 32,
 147,
 42,
 90,
 16,
 17,
 27,
 118,
 18,
 205,
 28,
 41,
 22,
 81,
 12,
 12,
 50,
 69,
 55,
 24,
 269,
 43,
 24,
 42,
 346,
 17,
 83,
 99,
 97,
 108,
 108,
 34,
 52,
 13,
 13,
 47,
 48,
 27,
 24,
 108,
 8,
 42,
 99,
 99,
 35,
 30,
 242,
 10,
 31,
 42,
 56,
 16,
 110,
 48,
 30,
 80,
 36,
 12,
 158,
 18]

In [61]:
from gensim.models import doc2vec

In [1]:
# def doc2Vec(df):
#     sentences = []
#     for index in range(df.shape[0]):
#         sentence = dataClean(df['text'][index])
# #         print(sentence)
#         sentences.append(sentence)
# #     newSentences=word2vec.Text8Corpus(sentences)
# #     model = doc2vec(sentences, min_count=1)
# #     print(sentences)
    
#     model = doc2vec.Doc2Vec(sentences)
#     return model
# #     model=word2vec.Word2Vec(sentences, size=50)
# #     words = list(model.wv.vocab)
# #     return model
# #     return model['sentences']
# doc2Vec(df)